# Нормализация 

In [ ]:
# Нормализация 
import re
import pymorphy2
from nltk.tokenize import word_tokenize

# Инициализация
morph = pymorphy2.MorphAnalyzer()

# Текст
text = "В 2024 году я купил 2 новых iPhone за 100000 рублей!!!"

# 1. Убрать числа и спецсимволы
clean_text = re.sub(r'[^а-яА-Яa-zA-Z\s]', ' ', text)  # оставляем только буквы и пробелы

# 2. Перевести в нижний регистр
clean_text = clean_text.lower()

# 3. Разбить на токены
tokens = word_tokenize(clean_text, language='russian')

# 4. Нормальные формы
normalized_tokens = [morph.parse(token)[0].normal_form for token in tokens if token.strip()]

print(f"Исходный текст: {text}")
print(f"Токены: {tokens}")
print(f"Нормальные формы: {normalized_tokens}")

def normalize_text(text):
    # Убрать всё кроме букв и пробелов
    text = re.sub(r'[^а-яА-Яa-zA-Z\s]', ' ', text)
    # В нижний регистр
    text = text.lower()
    # Разбить на слова
    words = text.split()
    # Нормальные формы (если pymorphy2 доступен)
    if 'pymorphy2' in globals():
        return [morph.parse(w)[0].normal_form for w in words]
    return words

normalized = normalize_text("Привет, как дела? 2024 год!")
print(normalized)

In [ ]:
import re
import pymorphy2

text = "Пример текста 2024!!!"
morph = pymorphy2.MorphAnalyzer()

# Все в одной строке
normalized = [morph.parse(word)[0].normal_form for word in 
              re.sub(r'[^а-яА-Яa-zA-Z\s]', ' ', text).lower().split() if word]

print(normalized)

## Векторизация

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from gensim.models import Word2Vec
import numpy as np

# Пример текстов
texts = ["машинное обучение это круто", 
         "глубокое обучение нейронные сети", 
         "машинное и глубокое обучение"]

# 1. CountVectorizer (мешок слов)
count_vectorizer = CountVectorizer(
    min_df=1,      # минимальная частота слова
    max_df=1.0,    # максимальная частота (1.0 = 100%)
    max_features=1000,  # максимальное число признаков
    ngram_range=(1, 3)  # униграммы, биграммы, триграммы
)
X_count = count_vectorizer.fit_transform(texts)
print(f"CountVectorizer: {X_count.shape}")

# 2. TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    min_df=1,
    max_df=1.0,
    max_features=1000,
    ngram_range=(1, 2)  # униграммы + биграммы
)
X_tfidf = tfidf_vectorizer.fit_transform(texts)
print(f"TF-IDF: {X_tfidf.shape}")

# 3. Word2Vec
# Подготовка токенизированных текстов
tokenized_texts = [text.split() for text in texts]

# Обучение Word2Vec
word2vec_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,      # размер вектора
    window=5,            # окно контекста
    min_count=1,         # минимальная частота слова
    workers=4
)

# Вектор для всего документа (среднее векторов слов)
def doc_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_word2vec = np.array([doc_vector(tokens, word2vec_model) for tokens in tokenized_texts])
print(f"Word2Vec: {X_word2vec.shape}")

# Быстрая проверка
print("\nПараметры и результаты:")
print(f"Слова CountVectorizer: {count_vectorizer.get_feature_names_out()[:5]}")
print(f"Слова TF-IDF: {tfidf_vectorizer.get_feature_names_out()[:5]}")
print(f"Слова Word2Vec: {list(word2vec_model.wv.index_to_key)[:5]}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF с настройками
vectorizer = TfidfVectorizer(
    min_df=2,           # игнорировать слова встречающиеся <2 раз
    max_df=0.9,         # игнорировать слова встречающиеся >90% документов
    max_features=5000,  # максимум 5000 признаков
    ngram_range=(1, 2)  # слова и пары слов
)

X = vectorizer.fit_transform(texts)
print(f"Размерность: {X.shape}")

In [ ]:
# Важные параметры:
# min_df=2 - убрать редкие слова (шум)
# max_df=0.9 - убрать слишком частые слова (стоп-слова)
# max_features=5000 - ограничить размерность
# ngram_range=(1, 3) - учитывать словосочетания

vectorizer = TfidfVectorizer(
    min_df=2,
    max_df=0.9, 
    max_features=5000,
    ngram_range=(1, 3),
    stop_words='russian'  # удалить стоп-слова
)

In [ ]:
# Быстрый Word2Vec (если текстов много)
from gensim.models import Word2Vec

# Токенизированные тексты
tokenized = [text.split() for text in texts]

# Обучение
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=1)

# Вектор документа = среднее векторов слов
doc_vectors = [np.mean([model.wv[word] for word in doc if word in model.wv], axis=0) 
               for doc in tokenized]

In [ ]:
# CountVectorizer - частота слов
# TF-IDF - важность слов в документах  
# Word2Vec - семантические векторы

# Выбор:
# 1. Для классификации/кластеризации → TF-IDF
# 2. Для семантического поиска → Word2Vec
# 3. Для быстрого базлайна → CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, max_df=0.9, ngram_range=(1, 2))
X = vectorizer.fit_transform(texts)
print(f"Признаков: {X.shape[1]}")

## Семантика

весь

In [ ]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# Пример текстов
texts = ["машинное обучение и глубокое обучение", 
         "нейронные сети и искусственный интеллект",
         "анализ данных и машинное обучение"]

# 1. Популярные словосочетания (n-граммы)
vectorizer = CountVectorizer(ngram_range=(2, 3), max_features=50)  # биграммы и триграммы
X = vectorizer.fit_transform(texts)

# Считаем частоты
ngram_counts = X.sum(axis=0).A1
ngram_names = vectorizer.get_feature_names_out()

# Топ-10 популярных словосочетаний
top_ngrams = sorted(zip(ngram_names, ngram_counts), key=lambda x: x[1], reverse=True)[:10]
print("Топ-10 популярных словосочетаний:")
for ngram, count in top_ngrams:
    print(f"  {ngram}: {count}")

# 2. Тематическое моделирование (LDA)
# Сначала создаем матрицу "термины-документы"
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
tf = tf_vectorizer.fit_transform(texts)

# LDA модель
n_topics = 3  # количество тем
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42
)

# Обучение
lda.fit(tf)

# Вывод тем
print(f"\nТемы (LDA, n_topics={n_topics}):")
feature_names = tf_vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"\nТема #{topic_idx}:")
    # Топ-10 слов для темы
    top_words_idx = topic.argsort()[:-11:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"  {' '.join(top_words)}")

# Распределение тем по документам
doc_topic_dist = lda.transform(tf)
print(f"\nРаспределение тем по документам:")
for i, dist in enumerate(doc_topic_dist):
    print(f"Документ {i}: {dist.round(3)}")

Кратко

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# LDA быстро
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts)

lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)

# Топ-слова для каждой темы
for i, topic in enumerate(lda.components_):
    print(f"Тема {i}:", ' '.join([vectorizer.get_feature_names_out()[j] 
                                  for j in topic.argsort()[-10:]]))

Популярные словосочетания быстро:

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# Биграммы
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
X_bigram = bigram_vectorizer.fit_transform(texts)

# Топ-5 биграмм
bigram_counts = X_bigram.sum(axis=0).A1
top_bigrams = sorted(zip(bigram_vectorizer.get_feature_names_out(), bigram_counts), 
                     key=lambda x: x[1], reverse=True)[:5]
print("Топ биграммы:", top_bigrams)

LDA с настройками:

In [ ]:
# Важные параметры LDA:
lda = LatentDirichletAllocation(
    n_components=5,      # число тем
    max_iter=10,         # итераций
    learning_method='online',  # для больших данных
    random_state=42,
    n_jobs=-1            # параллельные вычисления
)

Визуализация тем:

In [ ]:
import matplotlib.pyplot as plt

# Визуализация распределения тем для документа
doc_idx = 0
topic_dist = lda.transform(X)[doc_idx]

plt.bar(range(len(topic_dist)), topic_dist)
plt.xlabel('Тема')
plt.ylabel('Вероятность')
plt.title(f'Распределение тем для документа {doc_idx}')
plt.show()

Все в 4 строках:

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X = vectorizer.fit_transform(texts)
lda = LatentDirichletAllocation(n_components=3).fit(X)

# Темы
for i, topic in enumerate(lda.components_):
    print(f"Тема {i}:", ' '.join(vectorizer.get_feature_names_out()[topic.argsort()[-5:]]))

n-граммы → для поиска устойчивых словосочетаний

LDA → для тематического анализа

max_features в CountVectorizer → ограничивает словарь

n_components в LDA → подбирается экспериментально



## Визуализация текстовых данных

весь

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder

# Пример данных
texts = ["машинное обучение", "глубокое обучение", "нейронные сети", 
         "футбол матч гол", "баскетбол мяч кольцо", "теннис ракетка"]
labels = ["ai", "ai", "ai", "sport", "sport", "sport"]  # классы

# 1. Векторизация текстов
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts)

# 2. Уменьшение размерности до 2D
pca = PCA(n_components=2)
X_2d = pca.fit_transform(X.toarray())

# 3. Визуализация по классам
le = LabelEncoder()
y_encoded = le.fit_transform(labels)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_encoded, cmap='viridis', s=100, alpha=0.7)

# Подписи точек
for i, txt in enumerate(texts[:6]):  # первые 6 для наглядности
    plt.annotate(txt[:10], (X_2d[i, 0], X_2d[i, 1]), fontsize=9)

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Визуализация текстов (PCA)')
plt.colorbar(scatter, label='Классы')
plt.grid(True, alpha=0.3)
plt.show()

# 4. 3D визуализация
from mpl_toolkits.mplot3d import Axes3D

pca_3d = PCA(n_components=3)
X_3d = pca_3d.fit_transform(X.toarray())

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter_3d = ax.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], 
                        c=y_encoded, cmap='viridis', s=100, alpha=0.7)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('3D визуализация текстов')
plt.show()

# 5. Облако слов для каждого класса
all_text = ' '.join(texts)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Общее облако слов')

# Облако слов по классам
plt.subplot(1, 2, 2)
class_texts = {
    'AI': ' '.join([t for t, l in zip(texts, labels) if l == 'ai']),
    'Sport': ' '.join([t for t, l in zip(texts, labels) if l == 'sport'])
}

for idx, (cls, txt) in enumerate(class_texts.items()):
    wc = WordCloud(width=400, height=200, background_color='white').generate(txt)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Класс: {cls}')

plt.tight_layout()
plt.show()

кратко

In [ ]:
# PCA визуализация текстов
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Векторизация и PCA
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts)
X_2d = PCA(n_components=2).fit_transform(X.toarray())

# График
plt.scatter(X_2d[:, 0], X_2d[:, 1], c=labels)
plt.show()

# Облако слов
from wordcloud import WordCloud
wordcloud = WordCloud().generate(' '.join(texts))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

3д

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

X_3d = PCA(n_components=3).fit_transform(X.toarray())
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], c=labels)
plt.show()

Облако

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Для каждого класса свое облако
for label in set(labels):
    class_texts = [t for t, l in zip(texts, labels) if l == label]
    wordcloud = WordCloud().generate(' '.join(class_texts))
    plt.figure()
    plt.imshow(wordcloud)
    plt.title(f'Класс: {label}')
    plt.axis('off')
    plt.show()

TSNE НО ДОЛГИЙ

In [ ]:
from sklearn.manifold import TSNE

# TSNE лучше сохраняет локальные связи
X_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.toarray())
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=labels, alpha=0.7)
plt.title('TSNE визуализация текстов')
plt.show()

Коротко в 5 стр

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

X = TfidfVectorizer().fit_transform(texts)
X_2d = PCA(n_components=2).fit_transform(X.toarray())
plt.scatter(X_2d[:, 0], X_2d[:, 1], c=labels)
plt.show()

Правила:

PCA → быстрая линейная визуализация

TSNE → качественная нелинейная визуализация (медленнее)

Цвет точек = классы

Облако слов → частотность терминов

Размер точек можно сделать пропорциональным длине текста



Классификация текста

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Данные
texts = ["машинное обучение", "глубокое обучение", "нейронные сети", 
         "футбол матч гол", "баскетбол мяч кольцо", "теннис ракетка"]
labels = [0, 0, 0, 1, 1, 1]  # 0=AI, 1=Sport

# 1. Традиционные ML методы (после векторизации)
# TF-IDF + классификаторы
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\nRandom Forest:")
print(classification_report(y_test, y_pred_rf))

# 2. Нейросети с Keras Tokenizer
# Токенизация
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
X_seq = tokenizer.texts_to_sequences(texts)

# Паддинг до одинаковой длины
max_len = max(len(x) for x in X_seq)
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='post')

# Разделение для нейросети
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(
    X_pad, np.array(labels), test_size=0.3, random_state=42
)

# 3. Простая нейросеть
model_simple = keras.Sequential([
    layers.Embedding(input_dim=1000, output_dim=64, input_length=max_len),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # бинарная классификация
])

model_simple.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_simple.fit(X_train_nn, y_train_nn, epochs=10, batch_size=2, verbose=0)

# 4. Нейросеть с LSTM
model_lstm = keras.Sequential([
    layers.Embedding(input_dim=1000, output_dim=64, input_length=max_len),
    layers.LSTM(64, return_sequences=False),  # LSTM слой
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_train_nn, y_train_nn, epochs=10, batch_size=2, verbose=0)

# Оценка моделей
print("\nНейросети:")
print("Простая сеть - Accuracy:", model_simple.evaluate(X_test_nn, y_test_nn, verbose=0)[1])
print("LSTM сеть - Accuracy:", model_lstm.evaluate(X_test_nn, y_test_nn, verbose=0)[1])

Кратко

In [ ]:
# 1. TF-IDF + классификаторы
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"F1: {f1_score(y_test, y_pred):.3f}")

# 2. Нейросеть с LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Токенизация
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
X_seq = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=50)

# LSTM модель
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_seq, labels, epochs=5, verbose=0)

In [ ]:
# Би-LSTM (лучше для контекста)
model_bilstm = keras.Sequential([
    layers.Embedding(1000, 64),
    layers.Bidirectional(layers.LSTM(32)),  # двунаправленный LSTM
    layers.Dense(1, activation='sigmoid')
])

# Что использовать:
# 1. Если данных мало → TF-IDF + LogisticRegression/RandomForest
# 2. Если данных много → LSTM/Bi-LSTM
# 3. Если нужна скорость → TF-IDF + линейные модели
# 4. Если важен контекст → LSTM с Attention

# Параметры токенизатора:
# num_words=10000 - ограничить словарь
# max_len=100 - обрезать длинные тексты

6 стр

In [ ]:
# Традиционный ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X = TfidfVectorizer().fit_transform(texts)
model = LogisticRegression().fit(X, labels)
print("Accuracy:", model.score(X, labels))

# Нейросеть
model_lstm = keras.Sequential([layers.Embedding(1000, 64), layers.LSTM(64), layers.Dense(1, activation='sigmoid')])
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

многоклассовая

In [ ]:
# Для N классов
model = keras.Sequential([
    layers.Embedding(1000, 64),
    layers.LSTM(64),
    layers.Dense(num_classes, activation='softmax')  # softmax вместо sigmoid
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',  # другая функция потерь
              metrics=['accuracy'])